In [1]:
!nvidia-smi

Wed Jul 14 10:34:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# installing requirements

In [2]:
!pip install -U --pre tensorflow=="2.*"
!pip install tf_slim
!pip install pycocotools
!pip install tensorflow-object-detection-api

     |████████████████████████████████| 458.2MB 34kB/s 
     |████████████████████████████████| 471kB 36.6MB/s 
     |████████████████████████████████| 4.3MB 38.8MB/s 
     |████████████████████████████████| 1.3MB 35.5MB/s 
  Created wheel for clang: filename=clang-5.0-cp37-none-any.whl size=30696 sha256=84cbae9255ded989d43c55e815ff8147cea58cb8e7336229bdda0b6382441206
  Stored in directory: /root/.cache/pip/wheels/a1/d0/fb/f47e814ad88e0741b0ff6fc57bc4044f61c44cb707a68de6b3
Successfully built clang
  Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Found existing installation: grpcio 1.34.1
    Uninstalling grpcio-1.34.1:
      Successfully uninstalled grpcio-1.34.1
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3
  Found existing installation: tensorflow 2.5.0
    Uninstalling tensorflow-2.5.0:
      Successfu

# Imports

In [8]:
import numpy as np
import tensorflow as tf
import six.moves.urllib as urllib
import os
import sys
import tarfile
import zipfile
import pathlib
import imageio
import time

from matplotlib import pyplot as plt
from IPython.display import display
from collections import defaultdict
from io import StringIO
from PIL import Image


from object_detection.utils import ops as utils_ops
from object_detection.utils import visualization_utils as vis_util
from object_detection.utils import label_map_util

In [4]:
utils_ops.tf = tf.compat.v1
tf.gfile = tf.io.gfile

# Clone to Tensorflow github repasitory

In [5]:
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 2799, done.
remote: Counting objects: 100% (2799/2799), done.
remote: Compressing objects: 100% (2331/2331), done.
remote: Total 2799 (delta 719), reused 1318 (delta 433), pack-reused 0
Receiving objects: 100% (2799/2799), 32.78 MiB | 30.57 MiB/s, done.
Resolving deltas: 100% (719/719), done.


In [6]:
PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# Helper functions

In [7]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))

  return model


def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  input_tensor = tf.convert_to_tensor(image)
  input_tensor = input_tensor[tf.newaxis,...]

  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}

  output_dict['num_detections'] = num_detections

  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  if 'detection_masks' in output_dict:

    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict



def show_inference(model, image):

  output_dict = run_inference_for_single_image(model, image)
  
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)

  return image_np


# RUN

In [9]:
model_name = "mask_rcnn_inception_resnet_v2_atrous_coco_2018_01_28"
masking_model = load_model(model_name)


video_reader = imageio.get_reader('/content/pexels-kelly-lacy-5473765.mp4')
video_writer = imageio.get_writer('/content/mp4_new.mp4')

start = time.time()
for frames in video_reader:
    image_np = frames
    
    new_image = show_inference(masking_model,image_np)

    video_writer.append_data(new_image)
end = time.time()

print(f"Runtime of the program is {end - start}")

video_writer.close()

727400448/727390102 [==============================] - 4s 0us/step
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Runtime of the program is 3846.3103482723236
